In [1]:
import pandas as pd
# import time
# from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
# import requests
# from io import StringIO

# Realizando Web Scraping

In [2]:
URL = "https://awoiaf.westeros.org/index.php/List_of_characters"

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Acer\AppData\Local\Temp\ipykernel_21336\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
driver.get(URL)

# Obter nome dos personagens

In [ ]:
# Obter elemento usando Selenium a partir de sua Classe
characters = driver.find_elements(By.CLASS_NAME, 'mw-parser-output')

# Transformar elemento em texto
text = characters[0].text

# Dividir texto em linhas
rows = text.split('\n')

In [6]:
# Passar texto para Series
s = pd.Series(rows)

# Explorando os dados iniciais

In [7]:
s.head(5)

0    Contents: A B C D E F G H I J K L M N O P Q R ...
1                                                     
2    This page lists all characters from the A Song...
3                                                    A
4                                        A certain man
dtype: object

In [8]:
s.tail(5)

3817                                                 Zhea
3818                                       Zhoe Blanetree
3819                                             Zia Frey
3820    Zollo, called "Fat Zollo". A Dothraki sellswor...
3821                                        ↑ Back to top
dtype: object

In [9]:
# Remove espaços em branco no início e final das linhas em nossa Series
s = s.str.strip()

In [10]:
# Verificando se temos linhas vazias
s[pd.isnull(s)].sum()

0

Ao olhar para o conteúdo, vemos que temos ruído, tal como os títulos e referências dos personagens, um texto
para voltar ao topo da página, letras que indicam as iniciais dos nomes, etc. Vamos começar fazendo a limpeza
para determinar o nome dos personagens.

# Limpeza do nome dos personagens

In [11]:
# Remover cabeçalho
s = s.drop(s.index[:3])

# Os nomes estão sempre divididos dos títulos e referências por vírgula. Podemos então separar nossa Series
# em colunas e jogar num DataFrame usando a vírgula como separador.
s = s.str.split(',', expand=True)

# Deixando apenas a coluna que vamos utilizar de "nomes de personagens"
df = s.drop(columns=[1, 2, 3, 4, 5])

In [12]:
# Renomear a coluna que vamos usar
df.rename(columns={0: 'character_name'}, inplace=True)

# Remover letras que indicam o início de uma nova letra
df = df.loc[ (df['character_name'].str.len() > 1)]

# Remover "↑ Back to top"
df = df[df["character_name"].str.contains("↑ Back to top")==False]

df = df.reset_index(drop=True)

df

,character_name
0,A certain man
1,Abelar Hightower
2,Abelon
3,Addam of Duskendale
4,Addam Frey
...,...
3762,Zharaq zo Loraq
3763,Zhea
3764,Zhoe Blanetree
3765,Zia Frey


Ao final dessa primeira parte, obtivemos um dataset com os 3767 nomes "limpos".